In [1]:
import cv2
import numpy as np
from numpy.linalg import inv
import constraints
import detect_diff
import draw_dartboard
import board_config as cfg

import importlib
importlib.reload(constraints)
importlib.reload(detect_diff)
importlib.reload(draw_dartboard)
importlib.reload(cfg)

from constraints import SCORES
from detect_diff import getDifference
from draw_dartboard import draw_regions

import sys 
# print(sys.path)

MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15


Reading reference image :  resources/shomen_cropped.jpg
creating Score Description...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................Saving aligned image :  outputs/draw_board.jpg
Reading reference image :  resources/shomen_cropped.jpg
creating Score Description......................................................................................................................................................................................................................................

In [2]:
def alignImages(im1, im2):
 
  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
   
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
   
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)
   
  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)
 
  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]
 
  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)
   
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
   
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
 
  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
   
  return im1Reg, h
 
if __name__ == '__main__':

    # Read reference image
    refFilename = "resources/shomen_cropped_inside_eraced.jpg"
    print("Reading reference image : ", refFilename)
    imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)

    # Read image to be aligned
    imFilename = "resources/0111.2.jpg"
    print("Reading image to align : ", imFilename);  
    im = cv2.imread(imFilename, cv2.IMREAD_COLOR)

    print("Aligning images ...")
    # Registered image will be resotred in imReg. 
    # The estimated homography will be stored in h. 
    imReg, h = alignImages(im, imReference)

    h_inv = np.linalg.inv(h)

    # Write aligned image to disk. 
    outFilename = "outputs/aligned0111.1.jpg"
    print("Saving aligned image : ", outFilename); 
    cv2.imwrite(outFilename, imReg)

    # Print estimated homography
    print("Estimated homography : \n",  h)


Reading reference image :  resources/shomen_cropped_inside_eraced.jpg
Reading image to align :  resources/0111.2.jpg
Aligning images ...
Saving aligned image :  outputs/aligned0111.1.jpg
Estimated homography : 
 [[ 7.32286451e-01 -6.83339206e-02 -1.43938608e+02]
 [-1.23298981e-01  8.38732064e-01 -2.05728946e+01]
 [-5.02519994e-04  2.24377073e-04  1.00000000e+00]]


In [3]:
  # Use inverse homography
  h_inv = np.linalg.inv(h)
  height, width, channels = img_new.shape
  img_back_to_new = cv2.warpPerspective(img_bcg, h_inv, (width, height))

  # Write inv homo to disk.
  outFilename = "outputs/inv.homo.jpg"
  print("Saving detected difference image : ", outFilename)
  cv2.imwrite(outFilename, img_back_to_new)

NameError: name 'img_new' is not defined

In [5]:
    # Read background image
    bcgFilename = "resources/0111.0.jpg"
    print("Reading reference image : ", bcgFilename)
    img_bcg = cv2.imread(bcgFilename, cv2.IMREAD_COLOR)

    # Read new image
    newFilename = "resources/0111.2.jpg"
    print("Reading reference image : ", newFilename)
    img_new = cv2.imread(newFilename, cv2.IMREAD_COLOR)

    gray_bcg = cv2.cvtColor(img_bcg, cv2.COLOR_BGR2GRAY)
    gray_new = cv2.cvtColor(img_new, cv2.COLOR_BGR2GRAY)

    diff_thred, gray1, gray2, diff = getDifference(gray_bcg, gray_new)

    print(img_new.shape, diff_thred.shape)
    # apply mask
    dst = np.empty_like(img_new)
    dst[:,:,0] = cv2.bitwise_and(img_new[:,:,0], diff_thred)
    dst[:,:,1] = cv2.bitwise_and(img_new[:,:,1], diff_thred)
    dst[:,:,2] = cv2.bitwise_and(img_new[:,:,2], diff_thred)

    # dst = cv2.bitwise_and(diff, diff_thred)

    # Write applied mask to disk.
    outFilename = "outputs/applymask.jpg"
    print("Saving detected difference image : ", outFilename)
    cv2.imwrite(outFilename, dst)

    # Write detected difference to disk.
    outFilename = "outputs/diff.jpg"
    print("Saving detected difference image : ", outFilename)
    cv2.imwrite(outFilename, diff_thred)


Reading reference image :  resources/0111.0.jpg
Reading reference image :  resources/0111.2.jpg
uint8
(720, 1080)


error: OpenCV(4.1.2) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/median_blur.dispatch.cpp:283: error: (-215:Assertion failed) (ksize % 2 == 1) && (_src0.dims() <= 2 ) in function 'medianBlur'


In [52]:
# Mask dartboard
# Read reference image
refFilename = "resources/shomen_black_background.jpg"
print("Reading reference image : ", refFilename)
img_black_uncropped = cv2.imread(refFilename, cv2.IMREAD_COLOR)

# Read image to be aligned
imFilename = "resources/shomen_cropped.jpg"
print("Reading image to align : ", imFilename);  
im_white_cropped = cv2.imread(imFilename, cv2.IMREAD_COLOR)

for i in range(im_white_cropped[0, :, 0].size):
    for j in range(im_white_cropped[:, 0, 0].size):
        r = np.linalg.norm((i - cfg.CENTER[0], j - cfg.CENTER[1]))
        if r < cfg.DOUBLE_OUT_R:
            im_white_cropped[j, i, 0] = 0
            im_white_cropped[j, i, 1] = 0
            im_white_cropped[j, i, 2] = 0

print("Aligning images ...")
# Registered image will be resotred in imReg. 
# The estimated homography will be stored in h. 
imReg_wc2bu, h = alignImages(im_white_cropped, img_black_uncropped)

# imReg_wc2bu = draw_regions(imReg_wc2bu, 200, 100)

imReg_wc2bu_gray = cv2.cvtColor(imReg_wc2bu, cv2.COLOR_BGR2GRAY)

# Write aligned image to disk. 
outFilename = "outputs/aligned_wc2bu.jpg"
print("Saving aligned image : ", outFilename); 
cv2.imwrite(outFilename, imReg_wc2bu)


Reading reference image :  resources/shomen_black_background.jpg
Reading image to align :  resources/shomen_cropped.jpg
Aligning images ...
Saving aligned image :  outputs/aligned_wc2bu.jpg


True

In [9]:

# Read image
imFilename = "resources/shomen.jpg"
print("Reading image : ", imFilename);  
im_shomen = cv2.imread(imFilename, cv2.IMREAD_COLOR)
tx = 280
ty = 50
# im_shomen = draw_regions(im_shomen, tx, ty)
center = (cfg.CENTER[0] + tx, cfg.CENTER[1] + ty)
mask_shomen = im_shomen.copy()[:,:,0]
for i in range(im_shomen[0, :, 0].size):
    for j in range(im_shomen[:, 0, 0].size):
        r_in = np.linalg.norm((i - center[0], j - center[1]))
        # r_out = np.linalg.norm((i - center[0], j - center[1] - 70))
        if r_in > cfg.DOUBLE_OUT_R:
            # im_shomen[j, i, 0] = 0
            # im_shomen[j, i, 1] = 0
            # im_shomen[j, i, 2] = 0
            mask_shomen[j, i] = 0
        else: 
            mask_shomen[j, i] = 255
for t in [0, 30, 45, 60]:
    print(t)
    for i in range(im_shomen[0, :, 0].size):
        for j in range(im_shomen[:, 0, 0].size):
            r = np.linalg.norm((i - center[0] - 100*np.cos(t/180*np.pi), j - center[1] + 100*np.sin(t/180*np.pi)))
            if r < cfg.DOUBLE_OUT_R:
                mask_shomen[j, i] = 255

print(mask_shomen.shape)
# Write image to disk. 
outFilename = "outputs/mask_shomen_regions.jpg"
print("Saving image : ", outFilename); 
cv2.imwrite(outFilename, mask_shomen)


Reading image :  resources/shomen.jpg
0
30
45
60
(720, 1080)
Saving image :  outputs/mask_shomen_regions.jpg


True